In [1]:
# Implementation of the Eisenberg and Noe 2001 Debt Model in Python
# Eisenberg and Noe 2001 analyse the properties of intercorporate cash flows in financial 
# systems featuring cyclical interdependence and endogenously determined clearing vectors.
# The model computes clearing vectors for interlinked financial systems. A clearing vector is 
# a vector of payments from nodes in the financial system to other nodes and must satisfy the 
# conditions of proportional repayments of liabilities in default (we assume all debt claims have equal priority), limited liability of equity
# and absolute priority of debt over equity. The clearing vector is computed through a "fictitious 
# sequential default" algorithm in which the set of defaulting firms at the start of each round is fixed by
# the dynamic adjustments of the system from the preceding round. In each new round,an attempt to clear the 
# system that assumes that only nodes that defaulted in the previous round default. If no new defaults occur, 
# the algorithm terminates (i.e. a node is a distincy economig entity or a financial node i.e. a firm) 

# This algorithm gives the clearing vector and a natural measure of systemic risk (i.e. the exposure of a given node
# in the system to defaults by other firms - this is based on the number of waves of defaults required to 
# induce a given firm in the system to fail)

# Julian Kanjere, knjjul001@myuct.ac.za, September 2021
######### IMPORTS ######### 

import numpy as np

######### /IMPORTS ######### 

######### MODEL SETUP ######### 

NETWORK_TYPE = 'STAR' #'CIRCULANT'

NUM_AGENTS = 5 # i.e. n

# n x n nominal liabilities matrix L captures the nominal liability of one node to another in the system
# i.e. L_ij is nominal liability of node i to node j 
NOMINAL_LIABILITY_MATRIX = np.zeros((NUM_AGENTS, NUM_AGENTS)) 

# n x n relative liabilities matrix which represents the nominal liability of one node to another in the system
# as a proportion of the debtor nodes total liabilites i.e. L_ij / p_i 
RELATIVE_LIABILITY_MATRIX = np.zeros((NUM_AGENTS, NUM_AGENTS)) 

# a dictionary whose key is the round starting from 1 and value is a list of exogenous cash infusion to a 
# node i.e. from outside sources
OPERATING_CASH_FLOW_VECTOR = {} 

# a dictionary whose key is the round starting from 1 and value is a list or a set of 
# total payments made by each node to other nodes in the system i.e. p = (p_1, p_2, p_3, ...)
TOTAL_DOLLAR_PAYMENT_VECTOR = {}

# a dictionary whose key is the round starting from 1 and value is a list of payments 
# made by each node to other nodes in the system i.e. p = (p_bar_1, p_bar_2, p_bar_3, ...)
TOTAL_OBLIGATION_VECTOR = {} 

# a dictionary whose key is the round starting from 1 and value is a list of payments
CLEARING_PAYMENT_VECTOR = {} 

######### /MODEL SETUP ######### 

######### HELPER FUNCTIONS ######### 

def bool_limited_liability(i):
    '''Function to check that the limited liability for a node i holds i.e. the payment made by the node is
    less than or equal to the sum of the payments received by the node plus the exogenous operating cash flow 
    i.e. TOTAL_DOLLAR_PAYMENT_VECTOR (i.e. p_i) value is less than or equal to calculate_total_cash_flow_for_node()'''
    pass

def bool_proportional_repay():
    pass

def bool_debt_over_equity(i):
    '''Function to check the absolute priority rule for a node i i.e. either obligations are paid in full or all avaialable 
    cash flow (i.e. sum of the payments received by the node plus the exogenous operating cash flow) is paid to creditors for a node i holds i.e. the payment made by the node is
    less than or equal to the sum of the payments received by the node plus the exogenous operating cash flow
    i.e. first compare TOTAL_DOLLAR_PAYMENT_VECTOR value (i.e. p_i) to the TOTAL_OBLIGATION_VECTOR value (i.e. p_hat_i)
    to establish whether obligations are paid in full. If p_i < p_hat_i, then:
    if calculate_total_cash_flow_for_node() - proposed payment (you could think p_i) > 0, 
    you fail the absolute priority condition and advise that payment p_i should be = calculate_total_cash_flow_for_node()'''
    pass

def bool_check_defaults(round):
    '''Function to check whether there are any defaults in a given round. If not, algorithm can terminate'''
    pass

def bool_check_relative_liabilities_matrix():
    '''Function to sanity check that the sum of the proportions in the RELATIVE_LIABILITY_MATRIX add up to 1
    i.e. Further, all payments are made to some node in the system, therefore for all nodes, the sum the of the
    proportions should equal 1.'''
    pass

def bool_check_clearing_payment_vector(r):
    '''Function to sanity check clearing payment vector for a round r for each node i. 
    Each node pays a minimum of either calculate_total_cash_flow_for_node() or TOTAL_OBLIGATION_VECTOR value (i.e. p_hat_i).'''
    pass

def calculate_operating_cash_flow_for_node(i):
    '''Function to calculate operating cash flow to node i in a given round which is the exogenous operating
    cash flow received by node i'''
    pass

def calculate_total_cash_flow_for_node(i, operating_cash_flow, liabilities_received):
    '''Function to calculate total cash flow to node i in a given round which is liabilities received (endogenous) 
    plus operating cashflow (exogenous)
    '''
    total_cash_flow = operating_cash_flow + liabilities_received
    return total_cash_flow

def calculate_nominal_liabilities_out_for_node(i):
    '''Function to calculate nominal liabilities out for node i in a given round i.e. debtor node's total
    liabilities. These nominal liabilities represent the promised payments due to other nodes in the system. 
    This is represented in the nominal liabilities matrix NOMINAL_LIABILITY_MATRIX. 
    '''
    pass

def calculate_relative_liabilities_out_for_node(i):
    '''Function to calculate relative liabilities out for node i in a given round i.e. the nominal liability of one 
    node to another in the system as a proportion of the debtor node's total liabilities i.e. Pi_ij = L_ij/p_i.
    These nominal liabilities represent the promised payments due to other nodes in the system. 
    This is represented in the nominal liabilities matrix RELATIVE_LIABILITY_MATRIX. 
    '''
    pass

def calculate_total_equity_for_node(i, total_cash_flow, liabilities_out):
    '''function to calculate total equity of node i in a given round which is total cash flow (i.e. liabilities received 
    (endogenous) plus operating cashflow (exogenous)) minus liabilities out (i.e. nominal payments)
    '''
    return total_cash_flow - liabilities_out

def calculate_total_value_for_node(i, equity, liabilities_out):
    '''Function to calculate total value of node i in a given round which is debt plus equity'''
    return total_cash_flow - liabilities_out

def calculate_systemic_risk(r):
    '''Function to calculate systemic risk for each firm in a given round i.e. this is based on the number of 
    waves of defaults required to induce a given firm in the system to fail'''
    pass

def calculate_total_nominal_obligation_for_node(r, i):
    '''Function to calculate the total nominal obligation for a node i in a round r. i.e. for j = 1 upto n,
    calculate the sum of L_ij)'''
    pass

def calculate_total_obligation_vector(r):
    '''Function to return the total obligation vector made of the obligations for each node
    in a round r. i.e. p_bar = (p1_bar, p2_bar,...,pn_bar). This vector represents the payment level required for 
    complete satisfaction of all contractual liabilities by all nodes. This will loop over all n nodes and
    return the output from calculate_total_nominal_obligation_for_node()'''
    pass

def calculate_operating_cashflow_vector(r):
    '''Function to return the operating cashflow vector comprised of exogenous operating cash flow
    received by each node in a given round r'''
    pass

def calculate_clearing_payment_vector(r):
    '''Function to return the clearing payment vector in a given round r. For each node i, we maximise the payment p
    which is the range [0, TOTAL_OBLIGATION_VECTOR i.e. p_hat_i] subject to p <= calculate_total_cash_flow_for_node().
    We then need to confirm that limited_liability and absolute priority are satisfied.'''
    pass

def update_nominal_liabilities_matrix(r):
    '''Function to initialise or update the nominal liabilities matrix L in a given round r. All nominal claims are 
    nonnegative (i.e. L_ij > 0) and no node has a nominal claim against itself (i.e. L_ii = 0).'''
    pass

def update_relative_liabilities_matrix(r):
    '''Function to initialise or update the relative liabilities matrix Pi in a given round r. This matrix captures
    the nominal liability of one node to another in the system as a proportion of the debtor node's total
    liabilities. After this is updated, you can call bool_check_relative_liabilities_matrix() to sanity check
    that the entries add up to 1'''
    pass
######### /HELPER FUNCTIONS ######### 

######### ALGORITHM ######### 
# Determine each node's payout assumming all other nodes meet their obligations

# If, under the assumption that all nodes pay fully, it is, in fact, the case that all obligations are satisfied, 
# then terminate the algorithm.

# If some nodes default even when all other nodes pay, try to solve the system again, assuming that only these 
# "first-order" defaults occur.

# If only first-order defaults occur under the new clearing vector, then terminate the algorithm.

# If second-order defaults occur, then try to clear again assuming only second-order defaults occur, and so on.

# It is clear that since there are only n nodes, this process must terminate after n iterations. The point at which 
# a node defaults under the algorithm is a measure of the node's exposure to the systemic risks faced by the 
# clearing system.

######### ALGORITHM ######### 

